## 2. Ensemble-based models to predict tissue
We are interested in predicting the tissue of origin using the latent spaces generated before.

Using samples projected into the different latent spaces, use at least two ensemble approaches to predict their tissue of origin:
One approach must use a learner/estimator that tends to overfit data.
Another approach must use a weak learner that slightly outperforms a random estimator.
Use a model evaluation strategy that allows you to play with different hyperparameters to select the best model, assess whether the models are good predictors across all tissues (take a look at the classification_reportLinks to an external site. function in sklearn) and whether they generalize well on unseen data.

### Load data and packages

### Dimension Reduction

#### PCA

#### UMAP

### Ensemble Modeling

#### Weak Learners: Boosting 

#### Strong Learners: Bagging